<a href="https://colab.research.google.com/github/afhamash/Estimators/blob/main/EstimatorsMLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import cvxpy as cp
import scipy as sp
import random
import time
import functools as ft  # needed from defining the function multiKron()
from matplotlib import pyplot as plt

from numpy import linalg as la
from scipy import linalg as sla
from scipy import optimize as opt
from scipy.optimize import minimize
from scipy.linalg import sqrtm as sqm
from numpy import trace as tr
from numpy import allclose as ac
from numpy.linalg import eigh as eigh
from numpy.linalg import norm as norm
from collections import Counter
from math import comb


rng = np.random.default_rng()
from numpy.linalg import norm
from scipy.stats import unitary_group as Unitary
from matplotlib.colors import TwoSlopeNorm
from matplotlib import rc
rc('animation', html='jshtml')
from sklearn.metrics import auc
from scipy.stats import bernoulli

# General definitions

In [5]:
# Initialize ggplot style for plots
plt.style.use('ggplot')
from matplotlib import rc
rc('animation', html='jshtml')

In [6]:
%matplotlib inline
from IPython.display import HTML

In [7]:
np.set_printoptions(precision=4)

In [8]:
# Define Pauli matrices
PI = np.array([[1, 0],[0, 1]], dtype = 'complex128')
PX = np.array([[0, 1],[1, 0]], dtype = 'complex128')
PY = np.array([[0, 0 - 1j],[0 + 1j, 0]], dtype = 'complex128')
PZ = np.array([[1, 0],[0, -1]], dtype = 'complex128')

#Defined as a tuple since pauli matrices need not be changed later on
Paulis = (PI, PX, PY, PZ)

In [24]:
# Returns a complex Positive semidefinite matrix.
def PSDMatrix(Dim, Rank):

  X = 2*np.random.randn(Dim, Rank) - 1 + 1j*(2*np.random.randn(Dim, Rank) - 1)
  P = X@(X.conj().T)

  return P


def DensityMatrix(Dim, Rank, IsReal = 0):

  if IsReal == 1:
    X = np.random.randn(Dim, Rank)
  else:
    X = np.random.randn(Dim, Rank) + 1j*(np.random.randn(Dim, Rank))

  #Multiply by its transpose to obtain a postive semidefinite matrix
  P = X@(X.conj().T)

  #Normalise to obtain density matrix
  P = P/np.trace(P)

  return P
  # Returns a real valued density matrix.


# Return an n dimensional probability vector
def ProbVec(n):
  p = np.random.rand(n)
  return p/np.sum(p)


# Faster way of computing trace inner product
def TrInnerProduct(A,B):
  return float(np.real(np.dot(A.flatten().conj(), B.flatten().T)))

TIP = TrInnerProduct


def TraceDistance(Rho, Sigma):
  return la.norm(Rho - Sigma, 'nuc')/2


def Fidelity(Rho, Sigma):
  return tr(sqm(Rho@Sigma))**2
  # SqRho = sqm(Rho)
  # return tr(sqm(SqRho@Sigma@SqRho))

In [25]:
def list_base4(number):
    if number == 0:
      yield 0
    else:
      while number > 0:
        number, digit = divmod(number, 4)
        yield digit


In [26]:
def stringToPauliString(s):

  P = 1 + 0j
  for i in s:
    P = np.kron(P, Paulis[int(i)])

  return P

In [27]:
def multiKron(*matrices):

  try:
    if len(matrices) == 1:
      matrices = matrices[0]
    M = ft.reduce(np.kron, matrices)
    return M
  except TypeError:
    raise Exception('multiKron() must recieve at least 2 matrices as input.')

In [96]:
def generatePauliBasis(n):

  temp = [list(list_base4(i)) for i in list(range(4**n))]
  # gets a list of base4 representation of every number from 0 to n-1

  base4Lists = [reversed(l) for l in temp] # reverse to get the correct ordering
  base4Strings = []  # we want the string representation, not list representation.
  for l in base4Lists:
    s = [str(j) for j in l]
    base4Strings += [''.join(s)]  # convert to string format (from list)

  pauliNumberStrings = ['0'*(n-len(s))+s for s in base4Strings]  # append zeros to make every string of length n

  PauliMatrices = [stringToPauliString(s) for s in pauliNumberStrings]  # create the Pauli matrix strings from letter strings

  PauliArray = [np.reshape(P, newshape = (1, 2**(2*n))) for P in PauliMatrices]  # An array of the Pauli basis elements

  PauliMap = np.concatenate(PauliArray, axis = 0) # A d**2 x d**2 matrix whose rows correspond to the flattened Pauli matrices

  return (PauliMatrices, PauliMap)

In [135]:
class Pauli:

  def __init__(self, n):
    self.I = PI
    self.X = PX
    self.Y = PY
    self.Z = PZ
    self.elements, self.measurementMap = generatePauliBasis(n)


In [136]:
class RandomQuantumState:

  def __init__(self, dim, rank, isReal = 0):
    pauliMeasurementMap = generatePauliBasis(int(np.log2(dim)))[1]
    self.dim = dim
    self.rank = rank
    self.denMat = DensityMatrix(dim, rank, isReal)
    # self.blochVector = np.array((TIP(PX, self.denMat), TIP(PY, self.denMat), TIP(PZ, self.denMat)))
    self.blochVector = pauliMeasurementMap@self.denMat.flatten()  # Might need to normalize by a factor of sqrt(d)


  def expectation(self, Observable):
    return TIP(Observable, self.denMat)


class ToQuantumState:

  def __init__(self, densityMatrix):
    pauliMeasurementMap = generatePauliBasis(int(np.log2(dim)))[1]
    self.denMat = densityMatrix
    self.dim = int((np.shape(self.denMat)[0]))
    # self.rank = rank
    self.blochVector = pauliMeasurementMap@self.denMat.flatten()  # Might need to normalize by a factor of sqrt(d)

  def expectation(self, Observable):
    return TIP(Observable, self.denMat)

# Use this instead of RandomQuantumState or ToQuantumState
class QuantumState:

  def __init__(self, *args, dim, rank, isReal = 0):
    pauliMeasurementMap = generatePauliBasis(int(np.log2(dim)))[1]
    # print(args)
    if args == ():
      self.denMat = DensityMatrix(dim, rank, isReal)
      self.dim = dim
      self.rank = rank
    else:
      self.denMat = args[0]
      self.dim = int((np.shape(self.denMat)[0]))
    self.blochVector = pauliMeasurementMap@self.denMat.flatten()  # Might need to normalize by a factor of sqrt(d)

  def toRebitVector(self):
    if self.dim == 2:
      return np.array((TIP(PX, self.denMat), TIP(PZ, self.denMat)))
    else:
      raise Exception('State must be 2-dimensional for rebit vector')

  def expectation(self, Observable):
    return TIP(Observable, self.denMat)


In [138]:
# Generate m instances of QuantumState object
def GenQuantumStates(m, dim, rank):
  States = [QuantumState(dim = dim, rank = rank) for i in range(m)]
  return States

In [139]:
def PlotStates(blochVectors):

  fig, axs = plt.subplots(figsize = (5,5))
  axs.scatter(blochVectors[:,0], blochVectors[:,1],  color = 'blue')

  axs.set_xlim(-1.2, 1.2)
  axs.set_ylim(-1.2, 1.2)
  Circ = plt.Circle((0, 0), 1,fill = False, ec = 'black')
  axs.add_artist(Circ)
  axs.set_aspect(1)
  fig.show()